In [ ]:
# import the functions file
import useful_functions
# import coreNLP_function zzz



# import other python useful modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
import graphviz

# elmo stuff
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

from simple_elmo import ElmoModel


from sklearn import metrics
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

from functools import reduce
import pickle
import random


# some NLP modules
import spacy
from tqdm import tqdm
import re
import time

pd.set_option('display.max_colwidth', 200)
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import unidecode
import contractions
import re
from textblob import Word


In [ ]:
# Required package for scraping android reviews
import json
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [ ]:
# Required package for Mining Apple App's Review

from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get('https://sensortower.com/ios/US/twelve-app/app/yubo-make-new-friends/1038653883/review-history?selected_tab=reviews')

from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer


In [ ]:
# zzz
# module = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
# el_model = ElmoModel()
# el_model.load("")


In [ ]:
# Some reqquired module to be installed
# !pip install -qq google-play-scraper
# !pip install -qq -U watermark
# !pip install google-play-scraper
# !pip install contractions
# !pip install spacy
# !pip install textblob

### - Starts Scrape Android App Review

In [ ]:
# Get the app packages for some android applications whose reviews we want to scrap...
# More android packages for diffrent android apps can be added depending on the quantity of reviewsw we want to extract

app_packages = [
  'com.anydo',
  'com.todoist',
  'com.ticktick.task',
  'com.habitrpg.android.habitica',
  'cc.forestapp',
  'com.oristats.habitbull',
  'com.levor.liferpgtasks',
  'com.habitnow',
  'com.microsoft.todos',
  'prox.lab.calclock',
  'com.gmail.jmartindev.timetune',
  'com.artfulagenda.app',
  'com.tasks.android',
  'com.appgenix.bizcal',
  'com.appxy.planner'
]

In [ ]:
# Lets loop through the apps an get their info
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

# This is a function that prints the app's info
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

    
# This contains lots of information including the number of ratings, number of reviews and number of ratings for each score (1 to 5). 

# Lets test the print_json function
# print_json(app_infos[0])

In [ ]:
# Scraping the reviews
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)
    

    
# Lets get the length of app reviews
# len(app_reviews)

# Now, we save the review dataframes to a csv file which will be re-imported latter for further processing
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('datasets/android_reviews.csv', index=None, header=True)

### - Starts Scrape Apple App Review
 The reviews section for apple product  doesn't have a download button, so we use a Selenium web scraper to download the information instead. The packages have been imported in the beginning section of this file

In [ ]:
# Required Installs
#!pip install pystemmer
#!pip install sklearn

In [ ]:
# Helper function

def get_page():
    doc = BeautifulSoup(driver.page_source)
    rows = doc.select("tbody tr")

    datapoints = []
    for row in rows:
        cells = row.select("td")
        data = {
            'Country': cells[0].text.strip(),
            'Date': cells[1].text.strip(),
            'Rating': cells[2].select_one('.gold')['style'],
            'Review': cells[3].select_one('.break-wrap-review').text.strip(),
            'Version': cells[4].text.strip()
        }
        datapoints.append(data)
    return datapoints

In [ ]:
all_data = []
wait = WebDriverWait(driver, 5, poll_frequency=0.05)
while True:
    wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '.ajax-loading-cover')))

    results = get_page()    
    all_data.extend(results)

    next_button = driver.find_elements_by_css_selector(".btn-group .pagination")[1]
    if next_button.get_attribute('disabled'):
        break
    next_button.click()
    time.sleep(0.5)
    # Doesn't trigger fast enough!
    # wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.ajax-loading-cover')))
  
   

In [ ]:
# Lets save the reviews into a csv file
df_apple = pd.DataFrame(all_data)
df_apple.to_csv("datasets/apple_reviews.csv", index=None, header=True)



In [ ]:
# TExt Vectorization
# English stemmer from pyStemmer
stemmer = Stemmer.Stemmer('en')

analyzer = TfidfVectorizer().build_analyzer()

# Override CountVectorizer
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: stemmer.stemWords(analyzer(doc))

# Create a new StemmedCountVectorizer
vectorizer = StemmedTfidfVectorizer()
matrix = vectorizer.fit_transform(known.Review)

# Build a dataframe of words, purely out of curiosity
words_df = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
words_df.head(5)

### Lets import the kaggle, android and apple datasets

In [ ]:
android_df = pd.read_csv("datasets/android_reviews.csv")
android_df = android_df.dropna(subset=['Review'])
android_df = android_df.drop(columns=['reviewID', 'userName', 'userImage', 'score', 'replyContent'])

apple_df = pd.read_csv("datasets/apple_reviews.csv")
apple_df = apple_df.dropna(subset=['Review'])
apple_df = apple_df.drop(columns=['Ratings', 'Comments'])


kaggle_df = pd.read_csv("datasets/kaggle_reviews.csv")
kaggle_df = kaggle_df.dropna(subset=['Review'])
kaggle_df = kaggle_df.drop(columns=['App', 'Sentiment', 'Sentiment_polarity', 'Sentiment_Subjectivity', ''])

In [ ]:
# Reviews 
android_review = android_df['Reviews']
apple_review = apple_df['Reviews']
kaggle_review = kaggle_df['Reviews']

In [ ]:
# part of speech tagging
android_words = word_tokenize(android_df['Reviews'])
apple_words = word_tokenize(apple_df['Reviews'])
kaggle_words = word_tokenize(kaggle_df['Reviews'])

android_pos_tag = nltk.pos_tag(android_words)
apple_pos_tag = nltk.pos_tag(apple_words)
kaggle_pos_tag = nltk.pos_tag(kaggle_words)

# Removal of accented Characters
unaccented_string_android = unidecode.unidecode(android_df['Reviews'])
unaccented_string_apple = unidecode.unidecode(apple_df['Reviews'])
unaccented_string_kaggle = unidecode.unidecode(kaggle_df['Reviews'])



In [ ]:
porter_stemmer = PorterStemmer()
word_lemmatizer = WordNetLemmatizer()


stem_android_review = porter_stemmer.stem(android_df['Reviews'])
stem_apple_review = porter_stemmer.stem(apple_df['Reviews'])
stem_kaggle_review = porter_stemmer.stem(kaggle_df['Reviews'])


lemmatize_android_review = porter_stemmer.stem(android_df['Reviews'])
lemmatize_apple_review = porter_stemmer.stem(apple_df['Reviews'])
lemmatize_kaggle_review = porter_stemmer.stem(kaggle_df['Reviews'])


In [ ]:
#  print samples of the word stemming and lemmatizing
for w in word_tokenize(android_df['Reviews']):
#     print(f'Actual Word - {w}')
#     print(f'Stem - {porter_stemmer.stem(w)}')
#     print(f'Lemma - {word_lemmatizer.lemmatize(w)}\n')

for q in word_tokenize(apple_df['Reviews']):
#     print(f'Actual Word - {q}')
#     print(f'Stem - {porter_stemmer.stem(q)}')
#     print(f'Lemma - {word_lemmatizer.lemmatize(q)}\n')

for t in word_tokenize(kaggle_df['Reviews']):
#     print(f'Actual Word - {t}')
#     print(f'Stem - {porter_stemmer.stem(t)}')
#     print(f'Lemma - {word_lemmatizer.lemmatize(t)}\n')

In [ ]:
# Lowercasing
android_df['Reviews'] = android_df['Reviews'].str.lower()
apple_df['Reviews'] = apple_df['Reviews'].str.lower()
kaggle_df['Reviews'] = kaggle_df['Reviews'].str.lower()


# Remove special character
android_df['Reviews'] = re.sub(r"[^a-zA-Z0-9]"," ",android_df['Reviews'])
apple_df['Reviews'] = re.sub(r"[^a-zA-Z0-9]"," ",apple_df['Reviews'])
kaggle_df['Reviews'] = re.sub(r"[^a-zA-Z0-9]"," ",kaggle_df['Reviews'])


# Expanding  contractions
android_expanded_words = []   
apple_expanded_words = []   
kaggle_expanded_words = []   

for word in android_df['Reviews'].split():
  # using contractions.fix to expand the shortened words
  expanded_words.append(contractions.fix(word))  
   
expanded_text_android = ' '.join(expanded_words_android)

for word in apple_df['Reviews'].split():
  # using contractions.fix to expand the shortened words
  expanded_words.append(contractions.fix(word))  
   
expanded_text_apple = ' '.join(expanded_words_apple)

for word in kaggle_df['Reviews'].split():
  # using contractions.fix to expand the shortened words
  expanded_words.append(contractions.fix(word))  
   
expanded_text_kaggle = ' '.join(expanded_words_kaggle)


In [ ]:
# Function to remove stopword
all_stopwords = sp.Defaults.stop_words

def remove_stopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in text_tokens if not word in all_stopwords]
    return (" ").join(tokens_filtered)

android_tokens = remove_stopwords(android_df['Reviews'])
apple_tokens = remove_stopwords(apple_df['Reviews'])
kaggle_tokens = remove_stopwords(kaggle_df['Reviews'])

In [ ]:
# correct mispelled words
def correct_word_spelling(word):
    word = Word(word)
    result = word.correct()
    return(result)

corrected_android_review = android_df['Reviews']
corrected_apple_review = apple_df['Reviews']
corrected_kaggle_review = kaggle_df['Reviews']

In [ ]:
# here we combine the review columns for each dataframe and 
df['combined_review'] = df.apply(lambda row: ' '.join([str(android_df['Reviews']), 
                                            str(apple_df['Reviews']), 
                                            str(kaggle_df['Reviews'])]), axis=1)

# annotate the extracted features from text
df['char_count'] = df['combined_review'].apply(len)
df['word_count'] = df['combined_review'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)
df['punctuation_count'] = df['combined_review'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 
df['title_word_count'] = df['combined_review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df['upper_case_word_count'] = df['combined_review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
df['stopword_count'] = df['combined_review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words]))

# display the overview
# df[['char_count', 'word_count', 'word_density', 'punctuation_count', 'title_word_count', 'upper_case_word_count', 'stopword_count']].head(10)

In [ ]:
# function to generate sentence vector of the sentence

def vectorize_review(sentence):
    M = []
    for w in word_tokenize(sentence):
#     for w in word_tokenize(unicode(sentence, 'utf8')):
        if not w.isalpha():
            continue
        if w in embeddings_index:
            M.append(embeddings_index[w])
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

xtrain_vector = [sent2vec(x) for x in xtrain[:10]]
xtrain_vector = np.array(xtrain_vector)



In [ ]:
# create count vectorizer function to do sample topic modeling first
cvectorizer = CountVectorizer(min_df=4, max_features=4000, ngram_range=(1,2))
cvz = cvectorizer.fit_transform(df['text'])

# generate topic models using Latent Dirichlet Allocation
lda_model = LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20, random_state=42)
X_topics = lda_model.fit_transform(cvz)
n_top_words = 10
topic_summaries = []

# get topics and topic terms
topic_word = lda_model.components_ 
vocab = cvectorizer.get_feature_names()

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
#     print('Topic {}: {}'.format(i, ' | '.join(topic_words)))